In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import userdata
upstage_api_key = userdata.get('upstage_key')

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 113.2 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber aiohttp tqdm nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 104.6 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
import tarfile
import os

# 압축 파일 경로
tar_path = '/content/drive/MyDrive/PRC_A.tar'

# 저장할 폴더 경로
output_dir = '/content/drive/MyDrive/A_PRC'
os.makedirs(output_dir, exist_ok=True)

# 압축 파일 열기
with tarfile.open(tar_path, 'r') as tar:
    for member in tar.getmembers():
        # 조건: 파일이면서 PRC_ 폴더 내, '제21대' 포함, .pdf 확장자
        if member.isfile() and \
           member.name.startswith('PRC_') and \
           '제21대' in member.name and \
           member.name.endswith('.pdf'):

            # 파일 추출
            extracted_file = tar.extractfile(member)
            if extracted_file is not None:
                # 저장할 파일 경로
                filename_only = os.path.basename(member.name)
                save_path = os.path.join(output_dir, filename_only)

                # 저장
                with open(save_path, 'wb') as f_out:
                    f_out.write(extracted_file.read())

print("✅ 필터링된 PDF 파일만 저장 완료.")


✅ 필터링된 PDF 파일만 저장 완료.


# PDF 파일 텍스트 전문으로 전환 -> A~Z_PRC_text 폴더에 저장

In [ ]:
import os
import requests
import time
import math
import glob  # [신규 추가] 파일 목록을 가져오기 위한 라이브러리
from pypdf import PdfReader, PdfWriter
from google.colab import userdata # Colab Secrets를 사용하기 위한 라이브러리

# ----------------------------------------------------------------------------
# [기존과 동일] PDF를 2단 형식에 맞게 전처리하는 함수
# ----------------------------------------------------------------------------
def preprocess_pdf_for_two_columns(original_pdf_path):
    """
    PDF의 각 페이지를 왼쪽과 오른쪽 절반으로 나누어 새로운 PDF를 생성합니다.
    이렇게 하면 OCR이 왼쪽 단을 먼저 읽고, 그 다음 오른쪽 단을 읽게 됩니다.

    Args:
        original_pdf_path (str): 원본 2단 PDF 파일 경로.

    Returns:
        str: 전처리된 새 PDF 파일의 경로.
    """
    print(f"--- 2단 형식 처리를 위해 '{os.path.basename(original_pdf_path)}' 파일 전처리를 시작합니다. ---")

    reader = PdfReader(original_pdf_path)
    writer = PdfWriter()

    # 출력 파일 이름 설정
    base, ext = os.path.splitext(original_pdf_path)
    preprocessed_pdf_path = f"{base}_preprocessed.pdf"

    for page_num, page in enumerate(reader.pages):
        # 페이지의 너비와 높이 가져오기
        width = page.mediabox.width
        height = page.mediabox.height

        # 1. 왼쪽 절반 자르기
        # cropbox는 [lower-left-x, lower-left-y, upper-right-x, upper-right-y]
        page.cropbox.upper_right = (width / 2, height)
        page.cropbox.lower_left = (0, 0)
        writer.add_page(page)

        # 2. 오른쪽 절반 자르기
        # 원본 페이지를 다시 가져와서 작업해야 함 (cropbox가 수정되었기 때문)
        original_page_for_right = reader.pages[page_num]
        original_page_for_right.cropbox.lower_left = (width / 2, 0)
        original_page_for_right.cropbox.upper_right = (width, height)
        writer.add_page(original_page_for_right)

    # 전처리된 PDF 파일로 저장
    with open(preprocessed_pdf_path, "wb") as f:
        writer.write(f)

    original_pages = len(reader.pages)
    new_pages = len(writer.pages)
    print(f"▶ 전처리 완료: 원본 {original_pages}페이지가 {new_pages}페이지(왼쪽/오른쪽 분리)로 변환되었습니다.")
    print(f"▶ 임시 파일 '{os.path.basename(preprocessed_pdf_path)}' 생성 완료.")

    return preprocessed_pdf_path

# ----------------------------------------------------------------------------
# [기존과 동일] PDF 조각을 처리하는 API 호출 함수
# ----------------------------------------------------------------------------
def extract_text_from_pdf_chunk(file_path, timeout=1800):
    """
    (헬퍼 함수) PDF '조각' 파일에서 텍스트를 추출합니다.
    """
    try:
        api_key = userdata.get('upstage_key')
        if not api_key:
            print("\n오류: Colab Secrets에 'upstage_key'가 설정되지 않았습니다.")
            return None
    except userdata.SecretNotFoundError:
        print("\n오류: Colab Secrets에서 'upstage_key'를 찾을 수 없습니다.")
        return None

    url = "https://api.upstage.ai/v1/document-digitization"
    headers = {"Authorization": f"Bearer {api_key}"}

    try:
        with open(file_path, "rb") as f:
            files = {"document": (os.path.basename(file_path), f, "application/pdf")}
            data = {"model": "ocr"}
            response = requests.post(url, headers=headers, files=files, data=data, timeout=timeout)
            response.raise_for_status()

            result_json = response.json()
            return result_json.get("text")

    except requests.exceptions.Timeout:
        print(f"\n오류: '{os.path.basename(file_path)}' 처리 중 타임아웃 발생.")
        return None
    except requests.exceptions.HTTPError as http_err:
        print(f"\nHTTP 오류 발생: {http_err}")
        try:
            print(f"오류 내용: {response.json()}")
        except:
            print(f"오류 내용 (텍스트): {response.text}")
        return None
    except Exception as e:
        print(f"\n예상치 못한 오류 발생: {e}")
        return None

# ----------------------------------------------------------------------------
# [기존과 동일] PDF를 분할하고, 진행도를 표시하며, 결과를 합치는 함수
# ----------------------------------------------------------------------------
def process_large_pdf_in_chunks(pdf_file_path, output_txt_path, chunk_size=15):
    """
    대용량 PDF를 작은 조각으로 나누어 순차적으로 처리하고 진행도를 표시합니다.

    Args:
        pdf_file_path (str): 원본 PDF 파일 경로.
        output_txt_path (str): 결과를 저장할 텍스트 파일 경로.
        chunk_size (int): 한 번에 처리할 페이지 수. API 제한에 따라 조절 필요.
    """
    if not os.path.exists(pdf_file_path):
        print(f"오류: '{pdf_file_path}' 파일을 찾을 수 없습니다.")
        return

    try:
        reader = PdfReader(pdf_file_path)
        total_pages = len(reader.pages)
        total_chunks = math.ceil(total_pages / chunk_size)
        print(f"\n총 {total_pages} 페이지(분리 후)의 PDF를 {chunk_size} 페이지씩, {total_chunks}개의 조각으로 나누어 처리합니다.")

        all_extracted_text = []
        temp_chunk_files = [] # [신규] 생성된 임시 청크 파일 목록 관리

        for i in range(total_chunks):
            start_page = i * chunk_size
            end_page = min((i + 1) * chunk_size, total_pages)

            original_page_start = start_page // 2 + 1
            original_page_end = end_page // 2
            print(f"\n--- [진행도: {i+1}/{total_chunks}] 분리된 페이지 {start_page+1}-{end_page} 처리 중... (원본 기준 약 {original_page_start}-{original_page_end} 페이지) ---")

            writer = PdfWriter()
            # 임시 파일 이름이 겹치지 않도록 고유한 이름 생성
            temp_pdf_path = f"temp_chunk_{os.path.basename(output_txt_path)}_{i+1}.pdf"
            temp_chunk_files.append(temp_pdf_path) # 목록에 추가

            for page_num in range(start_page, end_page):
                writer.add_page(reader.pages[page_num])

            with open(temp_pdf_path, "wb") as temp_pdf_file:
                writer.write(temp_pdf_file)

            print(f"'{temp_pdf_path}' 파일로 API 요청을 보냅니다.")
            extracted_text = extract_text_from_pdf_chunk(temp_pdf_path)

            if extracted_text:
                all_extracted_text.append(extracted_text)
                print(f"▶ 성공: {len(extracted_text)}자 텍스트 추출 완료.")
            else:
                print(f"▶ 실패: 이 조각에서 텍스트를 추출하지 못했습니다. 다음 조각으로 넘어갑니다.")
                all_extracted_text.append(f"\n\n--- 페이지 {start_page+1}-{end_page} 처리 실패 ---\n\n")

            if i < total_chunks - 1:
                print("API 부하 감소를 위해 2초 대기합니다...")
                time.sleep(1)

        final_text = "\n".join(all_extracted_text)
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(final_text)

        print("\n========================================================")
        print(f"🎉 파일 변환 완료! 추출된 내용이 '{output_txt_path}' 파일에 저장되었습니다.")
        print("========================================================")

    except Exception as e:
        print(f"전체 프로세스 중 오류가 발생했습니다: {e}")
    finally:
        # [수정] 생성된 모든 임시 청크 파일 삭제
        print("--- 생성되었던 임시 PDF 조각 파일들을 삭제합니다. ---")
        for temp_file in temp_chunk_files:
            if os.path.exists(temp_file):
                os.remove(temp_file)

# ----------------------------------------------------------------------------
# [★★★★★ 수정된 부분 ★★★★★] 메인 실행 부분
# ----------------------------------------------------------------------------
def main():
    """메인 실행 함수"""
    # [수정] 입출력 폴더 경로 설정
    input_folder = '/content/drive/MyDrive/A_PRC'
    output_folder = '/content/drive/MyDrive/A_PRC_text'

    # [신규] 출력 폴더가 없으면 생성
    os.makedirs(output_folder, exist_ok=True)
    print(f"출력 폴더: '{output_folder}'를 확인/생성했습니다.")

    # [신규] 입력 폴더에서 모든 PDF 파일 목록 가져오기
    # '_preprocessed.pdf'로 끝나는 임시 파일은 제외
    pdf_files = [f for f in glob.glob(os.path.join(input_folder, '*.pdf')) if not f.endswith('_preprocessed.pdf')]

    if not pdf_files:
        print(f"'{input_folder}' 폴더에서 처리할 PDF 파일을 찾을 수 없습니다.")
        return

    total_files = len(pdf_files)
    print(f"총 {total_files}개의 PDF 파일을 처리합니다.")

    # 한 번에 처리할 페이지 수를 설정합니다.
    # 페이지가 2배로 늘어나므로, 'too many pages' 오류가 발생하면 이 값을 15 -> 10, 20 -> 10 등으로 더 낮춰보세요.
    # 예: CHUNK_SIZE = 30은 분리된 페이지 30개, 즉 원본 15페이지에 해당합니다.
    CHUNK_SIZE = 30

    # [신규] 각 PDF 파일에 대해 반복 작업 수행
    for i, original_pdf_path in enumerate(pdf_files):
        print(f"\n{'='*25} [{i+1}/{total_files}] 파일 처리 시작 {'='*25}")
        print(f"입력 파일: {os.path.basename(original_pdf_path)}")

        # [신규] 출력 텍스트 파일 경로 동적 생성
        base_name = os.path.basename(original_pdf_path)
        file_name_without_ext, _ = os.path.splitext(base_name)
        output_txt_path = os.path.join(output_folder, f"{file_name_without_ext}.txt")
        print(f"출력 파일: {os.path.basename(output_txt_path)}")

        # 만약 이미 변환된 txt 파일이 있다면 건너뛰기
        if os.path.exists(output_txt_path):
            print(f"▶ 이미 변환된 파일 '{os.path.basename(output_txt_path)}'이 존재하므로 건너뜁니다.")
            continue

        preprocessed_pdf_path = None
        try:
            # 1. 2단 PDF를 전처리하여 왼쪽/오른쪽이 분리된 새 PDF 생성
            preprocessed_pdf_path = preprocess_pdf_for_two_columns(original_pdf_path)

            # 2. 전처리된 PDF를 사용하여 텍스트 추출 실행
            process_large_pdf_in_chunks(preprocessed_pdf_path, output_txt_path, chunk_size=CHUNK_SIZE)

        finally:
            # 3. 모든 작업 완료 후 전처리 과정에서 생성된 임시 파일 삭제
            if preprocessed_pdf_path and os.path.exists(preprocessed_pdf_path):
                print(f"\n--- 작업 완료 후 임시 파일 '{os.path.basename(preprocessed_pdf_path)}'을(를) 삭제합니다. ---")
                os.remove(preprocessed_pdf_path)

    print(f"\n{'='*25} 모든 작업 완료 {'='*25}")
    print(f"총 {total_files}개의 PDF 파일 처리를 완료했습니다.")


if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.

총 150 페이지(분리 후)의 PDF를 30 페이지씩, 5개의 조각으로 나누어 처리합니다.

--- [진행도: 1/5] 분리된 페이지 1-30 처리 중... (원본 기준 약 1-15 페이지) ---
'temp_chunk_상임위원회 회의록_051009_제21대_제386회_제1차_20210428.txt_1.pdf' 파일로 API 요청을 보냅니다.
▶ 성공: 33209자 텍스트 추출 완료.
API 부하 감소를 위해 2초 대기합니다...

--- [진행도: 2/5] 분리된 페이지 31-60 처리 중... (원본 기준 약 16-30 페이지) ---
'temp_chunk_상임위원회 회의록_051009_제21대_제386회_제1차_20210428.txt_2.pdf' 파일로 API 요청을 보냅니다.
▶ 성공: 32701자 텍스트 추출 완료.
API 부하 감소를 위해 2초 대기합니다...

--- [진행도: 3/5] 분리된 페이지 61-90 처리 중... (원본 기준 약 31-45 페이지) ---
'temp_chunk_상임위원회 회의록_051009_제21대_제386회_제1차_20210428.txt_3.pdf' 파일로 API 요청을 보냅니다.
▶ 성공: 33837자 텍스트 추출 완료.
API 부하 감소를 위해 2초 대기합니다...

--- [진행도: 4/5] 분리된 페이지 91-120 처리 중... (원본 기준 약 46-60 페이지) ---
'temp_chunk_상임위원회 회의록_051009_제21대_제386회_제1차_20210428.txt_4.pdf' 파일로 API 요청을 보냅니다.
▶ 성공: 33612자 텍스트 추출 완료.
API 부하 감소를 위해 2초 대기합니다...

--- [진행도: 5/5] 분리된 페이지 121-150 처리 중... (원본 기준 약 6

# A~Z_PRC_TEXT -> JSON 파일로 요약본 생성

In [ ]:
import os
import json
import time
import re
import glob
import asyncio
import aiohttp
import random

# --- (프롬프트는 동일) ---
SYSTEM_PROMPT = """
[역할 정의]
당신은 대한민국 국회 회의록 '일부'를 분석하여, 발언자와 그의 발언을 핵심 내용으로 요약하는 전문 AI 어시스턴트입니다.

[작업 목표]
주어진 회의록 텍스트에서 '발언자'를 식별하고, 그의 발언 내용 전체를 1~2 문장의 핵심적인 내용으로 간결하게 '발언요약'으로 정리하여, 단 하나의 JSON 객체를 생성합니다.

[처리 규칙]
1.  **발언자 식별**: 텍스트 시작 부분에 있는 `○` 기호 뒤의 직책과 이름을 '발언자'로 지정합니다.
2.  **발언 내용 요약**: 주어진 텍스트 전체를 요약하여 '발언요약' 값을 생성합니다.
3.  **부가 정보 무시**: 괄호 `()` 안의 내용이나 페이지 번호 등은 요약에서 제외합니다.
4.  **출력 형식**:
    *   결과는 반드시 `{"발언자": "...", "발언요약": "..."}` 형태의 단일 JSON 객체여야 합니다. 리스트(`[]`)로 감싸지 마십시오.
"""
LLM_MODEL = "solar-pro"
API_URL = "https://api.upstage.ai/v1/chat/completions"

# ==============================================================================
# ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 이 부분이 수정되었습니다 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
# ==============================================================================
# 동시 요청 수를 더 보수적으로 설정하여 API Rate Limit에 걸릴 확률을 낮춥니다.
MAX_CONCURRENT_REQUESTS = 5  # <--- (수정 1) 10에서 5로 줄임
# ==============================================================================
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ 이 부분이 수정되었습니다 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲


def get_api_key():
    try:
        from google.colab import userdata
        return userdata.get('upstage_key')
    except (ImportError, ModuleNotFoundError):
        print("오류: Colab 환경이 아닙니다. API 키를 직접 코드에 입력하거나 환경 변수로 설정해주세요.")
        return None
    except userdata.SecretNotFoundError:
        print("오류: Colab Secrets에서 'upstage_key'를 찾을 수 없습니다.")
        return None

# ==============================================================================
# ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 이 함수가 수정되었습니다 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
# ==============================================================================
async def async_summarize_chunk(session, semaphore, api_key, text_chunk, chunk_index):
    if not text_chunk or not text_chunk.strip():
        return None

    # --- 재시도 및 백오프 로직을 위한 설정 ---
    MAX_RETRIES = 5
    INITIAL_BACKOFF = 3  # <--- (수정 2) 초기 대기 시간을 2초에서 3초로 늘림

    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    data = {
        "model": LLM_MODEL,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": text_chunk}
        ],
        "response_format": {"type": "json_object"},
        "temperature": 0.1,
    }

    async with semaphore:
        # --- (추가 3) API 요청 사이에 최소 간격을 두어 부하를 분산 ---
        # 0.5초 ~ 1.5초 사이의 무작위 시간만큼 대기합니다.
        # chunk_index가 1이 아닐 때만, 즉 첫 요청이 아닐 때만 대기
        if chunk_index > 1:
             await asyncio.sleep(0.5 + random.uniform(0, 1))

        for attempt in range(MAX_RETRIES):
            try:
                if attempt > 0:
                    print(f"--- [발언 {chunk_index}] API 요청 재시도 ({attempt + 1}/{MAX_RETRIES}) ---")
                else:
                    print(f"--- [발언 {chunk_index}] API 요청 시작 ---")

                async with session.post(API_URL, headers=headers, json=data, timeout=120) as response:
                    response.raise_for_status()
                    response_data = await response.json()
                    content_str = response_data['choices'][0]['message']['content']
                    parsed_json = json.loads(content_str)

                    if "발언자" in parsed_json and "발언요약" in parsed_json:
                        print(f"▶ [발언 {chunk_index}] 성공: '{parsed_json.get('발언자')}' 요약 완료.")
                        return parsed_json
                    else:
                        print(f"경고: [발언 {chunk_index}] 반환된 JSON에 필수 키가 없습니다. 내용: {parsed_json}")
                        return None

            except aiohttp.ClientResponseError as e:
                if e.status == 429 or e.status >= 500:
                    if attempt < MAX_RETRIES - 1:
                        backoff_time = INITIAL_BACKOFF * (2 ** attempt)
                        sleep_time = backoff_time + random.uniform(0, 1)
                        print(f"!! [발언 {chunk_index}] API 오류 (상태: {e.status}). {sleep_time:.2f}초 후 재시도합니다.")
                        await asyncio.sleep(sleep_time)
                        continue
                    else:
                        print(f"오류: [발언 {chunk_index}] 최대 재시도 횟수({MAX_RETRIES}) 도달 후에도 실패. (상태: {e.status})")
                else:
                    print(f"복구 불가능한 API 오류 [발언 {chunk_index}] (상태: {e.status}): {e}")
                    break
            except json.JSONDecodeError:
                print(f"JSON 파싱 실패 [발언 {chunk_index}]. 모델 응답이 유효하지 않습니다. 재시도하지 않습니다.")
                break
            except asyncio.TimeoutError:
                print(f"요청 시간 초과 [발언 {chunk_index}]. 재시도하지 않습니다.")
                break
            except Exception as e:
                print(f"요약 중 알 수 없는 오류 [발언 {chunk_index}]: {e}")
                if attempt < MAX_RETRIES - 1:
                    await asyncio.sleep(INITIAL_BACKOFF)
                else:
                    break

        return None
# ==============================================================================
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲ 이 함수가 수정되었습니다 ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
# ==============================================================================


# async_process_file, main 함수 등 나머지 코드는 모두 동일합니다.
async def async_process_file(input_path, output_path, session, semaphore):
    api_key = get_api_key()
    if not api_key:
        print("API 키가 없어 파일 처리를 건너뜁니다.")
        return

    try:
        with open(input_path, 'r', encoding='utf-8') as f:
            full_text = f.read()
    except FileNotFoundError:
        print(f"오류: 입력 파일 '{input_path}'를 찾을 수 없습니다.")
        return

    chunks = re.split(r'(?=○)', full_text)
    filtered_chunks = [chunk.strip() for chunk in chunks if chunk.strip() and chunk.startswith('○')]

    if not filtered_chunks:
        print(f"경고: '{os.path.basename(input_path)}' 파일에서 유효한 발언 단위를 찾지 못했습니다.")
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump([], f)
        return

    total_chunks = len(filtered_chunks)
    print(f"'{os.path.basename(input_path)}' 파일을 총 {total_chunks}개의 발언으로 분할하여 병렬 처리 시작...")

    tasks = [
        async_summarize_chunk(session, semaphore, api_key, chunk, i + 1)
        for i, chunk in enumerate(filtered_chunks)
    ]
    results = await asyncio.gather(*tasks)
    all_summaries = [res for res in results if res is not None]

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(all_summaries, f, ensure_ascii=False, indent=4)
        print("\n--------------------------------------------------------")
        print(f"🎉 파일 작업 완료! {len(all_summaries)}/{total_chunks}개의 요약이 '{os.path.basename(output_path)}'에 저장되었습니다.")
        print("--------------------------------------------------------")
    except Exception as e:
        print(f"결과 파일 저장 중 오류: {e}")

async def main():
    INPUT_FOLDER = '/content/drive/MyDrive/A_PRC_text'
    OUTPUT_FOLDER = '/content/drive/MyDrive/A_PRC_QA'
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    print(f"입력 폴더: {INPUT_FOLDER}")
    print(f"출력 폴더: {OUTPUT_FOLDER}")
    print(f"최대 동시 요청 수: {MAX_CONCURRENT_REQUESTS}\n")

    txt_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.txt')))
    files_to_process = []

    for input_path in txt_files:
        base_filename = os.path.basename(input_path)
        name_without_ext = os.path.splitext(base_filename)[0]
        output_path = os.path.join(OUTPUT_FOLDER, f"{name_without_ext}.json")
        if os.path.exists(output_path):
            print(f"이미 처리된 파일 (건너뛰기): {base_filename}")
        else:
            files_to_process.append((input_path, output_path))

    if not files_to_process:
        print("\n모든 파일이 이미 처리되었거나 처리할 파일이 없습니다.")
        return

    print(f"\n총 {len(files_to_process)}개의 신규 파일을 처리합니다.\n")

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    async with aiohttp.ClientSession() as session:
        file_processing_tasks = [
            async_process_file(in_path, out_path, session, semaphore)
            for in_path, out_path in files_to_process
        ]
        await asyncio.gather(*file_processing_tasks)

# --- 메인 실행 부분 (Colab/Jupyter 환경에 맞게 수정) ---
start_time = time.time()

# asyncio.run() 대신 await를 사용하여 이미 실행 중인 루프에서 main()을 실행합니다.
await main()

end_time = time.time()
print(f"\n✨✨✨ 모든 파일 처리가 완료되었습니다! (총 소요 시간: {end_time - start_time:.2f}초) ✨✨✨")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
▶ [발언 405] 성공: '위원장 김도읍' 요약 완료.
!! [발언 406] API 오류 (상태: 429). 3.03초 후 재시도합니다.
--- [발언 408] API 요청 시작 ---
▶ [발언 407] 성공: '위원장 김도읍' 요약 완료.
--- [발언 409] API 요청 시작 ---
▶ [발언 408] 성공: '산업통상자원부제1차관 장영진' 요약 완료.
▶ [발언 400] 성공: '소병철 위원' 요약 완료.
--- [발언 410] API 요청 시작 ---
▶ [발언 409] 성공: '위원장 김도읍' 요약 완료.
--- [발언 411] API 요청 시작 ---
--- [발언 412] API 요청 시작 ---
▶ [발언 410] 성공: '산업통상자원부 제 1차관 장영진' 요약 완료.
--- [발언 413] API 요청 시작 ---
--- [발언 406] API 요청 재시도 (2/5) ---
!! [발언 406] API 오류 (상태: 429). 6.03초 후 재시도합니다.
--- [발언 414] API 요청 시작 ---
▶ [발언 412] 성공: '산업통상자원부제1차관 장영진' 요약 완료.
▶ [발언 411] 성공: '위원장 김도읍' 요약 완료.
▶ [발언 413] 성공: '위원장 김도읍' 요약 완료.
--- [발언 415] API 요청 시작 ---
▶ [발언 414] 성공: '산업통상자원부 제 1차관 장영진' 요약 완료.
--- [발언 416] API 요청 시작 ---
--- [발언 418] API 요청 시작 ---
--- [발언 417] API 요청 시작 ---
!! [발언 417] API 오류 (상태: 429). 3.85초 후 재시도합니다.
▶ [발언 415] 성공: '위원장 김도읍' 요약 완료.
▶ [발언 416] 성공: '전문위원 박동찬' 요약 완료.
▶ [발언 418] 성공: '금융위원회 금융정책국장 이형주' 요약 완료.
--- [발언 419] API 요청 시작 ---
--- [발언

In [ ]:
import os
import json
import pandas as pd
import re
from tqdm import tqdm
import unicodedata  # ⭐️ 유니코드 정규화를 위해 추가

def create_single_file_dataframe(file_path):
    """
    지정된 단일 국회 회의록 JSON 파일을 읽어 Pandas DataFrame으로 만듭니다.
    '국회본회의 회의록'과 '국회본회의 의 회의록' 형식을 모두 처리합니다.
    macOS 등에서 발생하는 한글 자모 분리(NFD) 문제를 처리합니다.

    Args:
        file_path (str): 처리할 JSON 파일의 전체 경로.

    Returns:
        pandas.DataFrame: 해당 파일의 회의록 데이터가 포함된 DataFrame.
                           파일을 찾지 못하거나 처리할 데이터가 없으면 빈 DataFrame을 반환합니다.
    """
    original_filename = os.path.basename(file_path)

    # ⭐️ 파일명을 NFC(완성형) 형식으로 정규화합니다.
    filename = unicodedata.normalize('NFC', original_filename)

    all_data = []

    pattern = re.compile(
        r'^(?P<type>국회본회의( 의)? 회의록|상임위원회 회의록)_'
        r'(?P<id>\d{6})_'
        r'제(?P<assembly>\d+)대_'
        r'제(?P<session>\d+)회_'
        r'제(?P<round>\d+)차_'
        r'(?P<date>\d{8})\.json$'
    )

    # 정규화된 파일명으로 매치 시도
    match = pattern.match(filename)

    if not match:
        # 경고 메시지에는 원본 파일명을 보여줍니다.
        print(f"경고: 파일명 형식이 올바르지 않아 건너뜁니다: {original_filename}")
        return pd.DataFrame()

    file_info = match.groupdict()

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            utterances = json.load(f)

        for i, utterance in enumerate(utterances, 1):
            meeting_type = file_info['type'].replace(' 의', '')

            row_data = {
                '회의록ID': file_info['id'],
                '회의록종류': meeting_type,
                '회의일자': pd.to_datetime(file_info['date'], format='%Y%m%d'),
                '대수': f"제{file_info['assembly']}대",
                '회기': f"제{file_info['session']}회",
                '차수': f"제{file_info['round']}차",
                '발언순번': i,
                '발언자': utterance.get('발언자', None),
                '발언요약': utterance.get('발언요약', None)
            }
            all_data.append(row_data)

    except json.JSONDecodeError:
        print(f"경고: JSON 파싱 오류가 발생했습니다. 파일을 건너뜁니다: {original_filename}")
        return pd.DataFrame()
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {original_filename}, 오류: {e}")
        return pd.DataFrame()

    if not all_data:
        return pd.DataFrame()

    df = pd.DataFrame(all_data)

    column_order = [
        '회의록ID', '회의록종류', '회의일자', '대수', '회기', '차수',
        '발언순번', '발언자', '발언요약'
    ]

    return df[column_order]

def create_combined_dataframe_from_folder(folder_path):
    """
    지정된 폴더 내의 모든 JSON 회의록 파일을 읽어 하나의 Pandas DataFrame으로 결합합니다.

    Args:
        folder_path (str): JSON 파일들이 있는 폴더의 경로.

    Returns:
        pandas.DataFrame: 폴더 내 모든 회의록 데이터가 포함된 단일 DataFrame.
                           처리할 파일이 없으면 빈 DataFrame을 반환합니다.
    """
    if not os.path.isdir(folder_path):
        print(f"오류: '{folder_path}' 폴더를 찾을 수 없습니다.")
        return pd.DataFrame()

    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

    if not json_files:
        print(f"'{folder_path}' 폴더에 처리할 JSON 파일이 없습니다.")
        return pd.DataFrame()

    json_files.sort()

    print(f"총 {len(json_files)}개의 JSON 파일을 처리합니다...")

    all_dfs = []

    for filename in tqdm(json_files, desc="폴더 처리 중"):
        file_path = os.path.join(folder_path, filename)
        single_df = create_single_file_dataframe(file_path)

        if not single_df.empty:
            all_dfs.append(single_df)

    if not all_dfs:
        print("처리된 데이터가 없어 빈 데이터프레임을 반환합니다.")
        return pd.DataFrame()

    print("\n모든 파일 처리가 완료되었습니다. 데이터프레임을 하나로 결합합니다...")

    combined_df = pd.concat(all_dfs, ignore_index=True)

    return combined_df

# --- 메인 실행 부분 ---

# ⭐️ 여기에 JSON 파일이 들어있는 '폴더의 전체 경로'를 입력하세요. ⭐️
TEST_FOLDER_PATH = '/content/drive/MyDrive/A_PRC_QA'

# 폴더 내 모든 JSON 파일 처리 함수를 호출합니다.
minutes_df = create_combined_dataframe_from_folder(TEST_FOLDER_PATH)

# 생성된 DataFrame의 기본 정보와 일부 데이터를 출력하여 확인합니다.
if not minutes_df.empty:
    print("\n--- 최종 데이터프레임 생성 완료 ---")
    print(f"총 {len(minutes_df)}개의 발언 데이터가 생성되었습니다.")
    print("\n--- 데이터프레임 정보 ---")
    print(minutes_df.info())
    print("\n--- 상위 5개 데이터 ---")
    print(minutes_df.head())
    print("\n--- 하위 5개 데이터 ---")
    print(minutes_df.tail())

총 113개의 JSON 파일을 처리합니다...


폴더 처리 중: 100%|██████████| 113/113 [00:07<00:00, 14.34it/s]


모든 파일 처리가 완료되었습니다. 데이터프레임을 하나로 결합합니다...

--- 최종 데이터프레임 생성 완료 ---
총 50884개의 발언 데이터가 생성되었습니다.

--- 데이터프레임 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50884 entries, 0 to 50883
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   회의록ID   50884 non-null  object        
 1   회의록종류   50884 non-null  object        
 2   회의일자    50884 non-null  datetime64[ns]
 3   대수      50884 non-null  object        
 4   회기      50884 non-null  object        
 5   차수      50884 non-null  object        
 6   발언순번    50884 non-null  int64         
 7   발언자     50884 non-null  object        
 8   발언요약    50884 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 3.5+ MB
None

--- 상위 5개 데이터 ---
    회의록ID      회의록종류       회의일자    대수     회기    차수  발언순번  \
0  050480  국회본회의 회의록 2020-11-19  제21대  제382회  제12차     1   
1  050480  국회본회의 회의록 2020-11-19  제21대  제382회  제12차     2   
2  050480  국회본회의 회의록 20

In [ ]:
minutes_df['회의록종류'] = minutes_df['회의록종류'].str.replace(' 회의록', '', regex=False)

In [ ]:
minutes_df_unit = pd.read_csv("발언자.csv",encoding="utf-8")

In [ ]:
 minutes_df_unit.columns

Index(['발언자'], dtype='object')

In [ ]:
# -----------------------------------------------------------------
# 0. 비동기 처리를 위한 라이브러리 설치 (최초 1회 실행)
# -----------------------------------------------------------------
# !pip install -q pandas httpx "nest_asyncio" tqdm # 필요한 라이브러리 설치

import pandas as pd
import requests
import json
import time

# --- 비동기 처리를 위한 라이브러리 ---
import asyncio
import httpx
import nest_asyncio
from tqdm.asyncio import tqdm # 비동기용 tqdm
# ------------------------------------

# Google Colab 환경일 경우에만 실행
try:
    from google.colab import userdata
    from IPython.display import display
    # Colab/Jupyter 환경에서 asyncio 이벤트 루프 중첩 허용
    nest_asyncio.apply()
except ImportError:
    userdata = None # Colab이 아닌 환경에서는 userdata를 None으로 설정
    # display 함수가 없는 환경을 위한 대체 함수
    def display(df):
        print(df.to_string())
    pass

# -----------------------------------------------------------------
# 1. API 설정 및 키 로드
# -----------------------------------------------------------------
LLM_MODEL = "solar-pro"
API_URL = "https://api.upstage.ai/v1/chat/completions"
MAX_RETRIES = 5
CONCURRENT_REQUESTS = 5

INITIAL_DELAY = 1
BACKOFF_FACTOR = 2

UPSTAGE_API_KEY = None
try:
    if userdata:
        UPSTAGE_API_KEY = userdata.get('upstage_key')
        if not UPSTAGE_API_KEY:
            raise ValueError("API 키가 비어있습니다.")
        print("✅ Google Colab Secret에서 API 키를 성공적으로 불러왔습니다.")
    else:
        # Colab이 아닌 환경에서는 직접 키를 입력하거나 환경 변수를 사용하세요.
        # 예: UPSTAGE_API_KEY = "YOUR_API_KEY_HERE"
        if not UPSTAGE_API_KEY:
             raise ValueError("API 키가 설정되지 않았습니다.")
        print("✅ API 키를 성공적으로 설정했습니다.")

except Exception as e:
    print(f"🔴 오류: API 키를 로드하는 데 실패했습니다. ({e})")
    print("➡️ Google Colab을 사용 중이라면 왼쪽 '키' 아이콘 메뉴에서 'upstage_key'라는 이름으로 API 키를 추가했는지 확인하세요.")


# -----------------------------------------------------------------
# 2. 샘플 데이터프레임 생성
# -----------------------------------------------------------------

minutes_df_unit = minutes_df_unit

print("\n[입력 데이터 확인]")
display(minutes_df_unit)
print(f"\n[컬럼 정보 확인]\n{minutes_df_unit.columns}")
print("-" * 50)


# -----------------------------------------------------------------
# 3. LLM 프롬프트 정의
# -----------------------------------------------------------------
# -----------------------------------------------------------------
# 3. LLM 프롬프트 정의 (✨ 수정된 버전)
# -----------------------------------------------------------------
SYSTEM_PROMPT = """
당신은 한국 국회 회의록 데이터를 처리하는 전문가입니다. 주어진 '발언자' 텍스트를 '직책'과 '발언자' 두 개의 필드로 분리하는 작업을 수행합니다. 아래의 규칙과 예시를 엄격히 준수하여, 결과를 반드시 JSON 형식으로 반환해야 합니다.

**규칙:**

1.  **기본 원칙**: 문자열에서 2~4글자의 **순수한 한국어 이름**만 '발언자'로 정확히 식별합니다. 이름 앞의 소속(위원회 등)과 이름 뒤에 붙는 '위원', '의원', '장관' 같은 직위는 **모두 '직책' 필드에 포함**시켜야 합니다.
2.  **다수 발언자**: `·`, `,`, `등`, `외` 키워드로 여러 사람이 언급된 경우, 이름들을 쉼표(,)로 묶어 '발언자' 필드에 넣고, 나머지 설명(예: `의원 발의`, `외 255인`)은 '직책' 필드에 넣습니다.
3.  **단독 항목**: 이름만 있으면 '발언자'에, 직책/단체명만 있으면 '직책'에 기입하고 다른 필드는 null로 설정합니다.
4.  **설명 항목**: 특정인의 발언이 아닌, 투표 결과, 법률안 명칭, 참석자 명단 등 설명에 해당하는 항목은 전체 내용을 '직책'에 넣고 '발언자'는 null로 설정합니다.
5.  **데이터 정제**:
    - 문자열 앞의 `○` 기호는 무시하고 처리합니다.

**처리 예시:**

-   입력: `○과학기술정보방송통신위원회 정필모 위원`
-   출력: `{"직책": "과학기술정보방송통신위원회 위원", "발언자": "정필모"}`

-   입력: `행정안전위원회 임호선 위원`
-   출력: `{"직책": "행정안전위원회 위원", "발언자": "임호선"}`

-   입력: `이탄희ㆍ최강욱ㆍ김용민 의원`
-   출력: `{"직책": "의원", "발언자": "이탄희, 최강욱, 김용민"}`

-   입력: `법제사법위원장 김도읍`
-   출력: `{"직책": "법제사법위원장", "발언자": "김도읍"}`


**출력 형식**:
반드시 아래와 같은 JSON 형식으로만 응답해야 합니다.
`{"직책": "...", "발언자": "..."}`
"""

# -----------------------------------------------------------------
# 4. LLM 호출 및 파싱 함수 정의
# -----------------------------------------------------------------
async def async_split_speaker_info(client, text, semaphore):
    async with semaphore:
        if not isinstance(text, str) or not text.strip():
            return {"직책": None, "발언자": None}

        headers = {"Authorization": f"Bearer {UPSTAGE_API_KEY}", "Content-Type": "application/json"}
        data = {
            "model": LLM_MODEL,
            "messages": [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": text}],
            "temperature": 0.1
        }
        delay = INITIAL_DELAY
        for attempt in range(MAX_RETRIES):
            try:
                response = await client.post(API_URL, headers=headers, json=data, timeout=30)
                response.raise_for_status()
                content = response.json()['choices'][0]['message']['content']
                start_index, end_index = content.find('{'), content.rfind('}')
                clean_json_str = content[start_index : end_index + 1] if start_index != -1 and end_index != -1 else content
                json_response = json.loads(clean_json_str)
                return {"직책": json_response.get("직책"), "발언자": json_response.get("발언자")}
            except httpx.HTTPStatusError as e:
                if e.response.status_code == 429:
                    retry_after = e.response.headers.get("Retry-After")
                    wait_time = int(retry_after) if retry_after and retry_after.isdigit() else delay
                    print(f"API 속도 제한(429). {wait_time}초 후 재시도. (입력: {text[:20]}...)")
                    await asyncio.sleep(wait_time)
                    if not (retry_after and retry_after.isdigit()): delay *= BACKOFF_FACTOR
                elif e.response.status_code >= 500:
                    print(f"서버 오류({e.response.status_code}). {delay}초 후 재시도... (입력: {text[:20]}...)")
                    await asyncio.sleep(delay)
                    delay *= BACKOFF_FACTOR
                else:
                    print(f"클라이언트 오류({e.response.status_code}). 재시도 중단. (입력: {text[:20]}...)")
                    return {"직책": f"API 호출 실패 ({e.response.status_code})", "발언자": text}
            except (httpx.RequestError, json.JSONDecodeError, KeyError, IndexError) as e:
                print(f"{type(e).__name__} 발생 (재시도 {attempt + 1}/{MAX_RETRIES})...")
                if attempt < MAX_RETRIES - 1: await asyncio.sleep(delay)
                else: return {"직책": "최종 처리 실패", "발언자": text}
        return {"직책": "API 호출 최종 실패", "발언자": text}

# -----------------------------------------------------------------
# 5. 비동기 작업 실행 및 결과 병합을 위한 메인 함수 정의
# -----------------------------------------------------------------
async def main():
    if not UPSTAGE_API_KEY:
        print("\nAPI 키가 없어 작업을 진행할 수 없습니다.")
        return

    semaphore = asyncio.Semaphore(CONCURRENT_REQUESTS)
    async with httpx.AsyncClient() as client:
        tasks = [async_split_speaker_info(client, speaker, semaphore) for speaker in minutes_df_unit['발언자']]
        results = await tqdm.gather(*tasks, desc="발언자 분리 중")

    split_df = pd.json_normalize(results)
    final_df = minutes_df_unit.rename(columns={'발언자': '원본_발언자'})
    split_df.rename(columns={'발언자': '분리된_발언자'}, inplace=True)
    final_df = pd.concat([final_df.reset_index(drop=True), split_df.reset_index(drop=True)], axis=1)

    print("\n\n✅ 작업 완료!")
    print("\n✨ 최종 결과:")
    display_cols = ['원본_발언자', '직책', '분리된_발언자']
    display(final_df[display_cols])

    # 최종 데이터프레임을 반환하여 변수에 저장할 수 있도록 함
    return final_df

✅ Google Colab Secret에서 API 키를 성공적으로 불러왔습니다.

[입력 데이터 확인]


,발언자
0,노웅래 · 한준호 . 임종성 · 남인순 · 이규민 · 민형배 의원 발의
1,의장 박병석
2,의사국장 박태형
3,예산결산특별위원회 결산심사소위원장 국민의 힘 추경호 위원
4,○의장 박병석
...,...
2749,과학기술정보통신부제2차관 강도현
2750,국토교통부 제1차관 진현환
2751,국토교통부 주택정책관 김헌정
2752,기획재정부공공정책국장 강영규



[컬럼 정보 확인]
Index(['발언자'], dtype='object')
--------------------------------------------------


In [ ]:
# 위에서 정의한 main 함수를 실행하고, 결과를 final_df 변수에 저장합니다.
final_df = await main()

발언자 분리 중:   9%|▊         | 240/2754 [01:27<14:50,  2.82it/s]

API 속도 제한(429). 1초 후 재시도. (입력: 행정중심복합도시건설청차장 문성요...)


발언자 분리 중:   9%|▉         | 243/2754 [01:29<16:01,  2.61it/s]

API 속도 제한(429). 2초 후 재시도. (입력: 행정중심복합도시건설청차장 문성요...)


발언자 분리 중:   9%|▉         | 244/2754 [01:30<22:28,  1.86it/s]

...

발언자 분리 중: 100%|█████████▉| 2749/2754 [26:16<00:03,  1.45it/s]

API 속도 제한(429). 4초 후 재시도. (입력: 국토교통위원장대리 박정하...)
API 속도 제한(429). 4초 후 재시도. (입력: ○안호영 위원...)


발언자 분리 중: 100%|██████████| 2754/2754 [26:22<00:00,  1.74it/s]



✅ 작업 완료!

✨ 최종 결과:


,원본_발언자,직책,분리된_발언자
0,노웅래 · 한준호 . 임종성 · 남인순 · 이규민 · 민형배 의원 발의,의원 발의,"노웅래, 한준호, 임종성, 남인순, 이규민, 민형배"
1,의장 박병석,의장,박병석
2,의사국장 박태형,의사국장,박태형
3,예산결산특별위원회 결산심사소위원장 국민의 힘 추경호 위원,예산결산특별위원회 결산심사소위원장 국민의 힘 추경호 위원,추경호
4,○의장 박병석,의장,박병석
...,...,...,...
2749,과학기술정보통신부제2차관 강도현,과학기술정보통신부 제2차관,강도현
2750,국토교통부 제1차관 진현환,국토교통부 제1차관,진현환
2751,국토교통부 주택정책관 김헌정,국토교통부 주택정책관,김헌정
2752,기획재정부공공정책국장 강영규,기획재정부공공정책국장,강영규


In [ ]:
# 데이터프레임 병합
merged_df = minutes_df.merge(
    final_df[['원본_발언자', '직책', '분리된_발언자']],
    how='left',
    left_on='발언자',
    right_on='원본_발언자'
)


In [ ]:
merged_df[['회의록ID', '회의록종류', '회의일자', '대수', '회기', '차수', '발언순번','직책', '분리된_발언자', '발언요약']].to_csv("merged_A.csv", index=False, encoding='utf-8-sig')
